**Pergunta-Guia: O Brasil é um país desigual?**



> Projeto do primeiro módulo da disciplina de estatística para ciência de dados. Composto pela equipe: Ana Caroline, Carlos Eduardo, Claudio Cruz, Julio Sousa e Maria Eduarda.
Nesse projeto analisamos e cruzamos os dados dos estados e munícipios do Nordeste.



# Importação e cruzamento dos dados

In [ ]:
import pandas as pd

!gdown --id 1GQVxYHY9ouZvh_Jkbzt3sjnWHslxGqnX
!gdown --id 1OLg85S7vAr4MQomc_wcaQhRo6SLciSu0
!gdown --id 1G3Ll5LsIhsvpodjnKg6JE0KXbc_D-BSP


In [ ]:
df_pib = pd.read_csv("tabela5938_2020.csv.xz")
df_pop = pd.read_csv("tabela6579_2020.csv.xz")
df_dtb = pd.read_csv("RELATORIO_DTB_BRASIL_MUNICIPIO.csv")

In [ ]:
df_pib

In [ ]:
df_pop

In [ ]:
df_dtb

In [ ]:
df_aux1 = pd.merge(df_pib,df_pop,how='outer',left_on='Cód.',right_on='Cód.')
df_aux1 = df_aux1.drop(columns=['Município_y'])
df_aux1

In [ ]:
df_geral = pd.merge(df_aux1,df_dtb,how='left',left_on='Cód.',right_on='Código Município Completo')
df_geral

In [ ]:
df_geral.info()

In [ ]:
uf_ne=['Alagoas','Bahia','Ceará','Maranhão','Paraíba','Pernambuco','Piauí','Rio Grande do Norte','Sergipe']
df_nordeste = df_geral[df_geral['Nome_UF'].isin(uf_ne)]

df_nordeste.info()

In [ ]:
df_nordeste = df_nordeste.drop(columns=['Código Município Completo'])

# Cálculo de variáveis auxiliares

# Aplicação de medidas estatísticas

**Cálculo do PIB per capita dos municípios do Nordeste em reais**

In [ ]:
df_nordeste['PIB per capita'] = (df_nordeste['PIB (Mil Reais)'] * 1000) / df_nordeste['População (Pessoas)']

df_nordeste.info()


In [ ]:
df_nordeste_teste = df_nordeste.groupby(['Nome_UF'])['PIB per capita']

In [ ]:
df_nordeste_teste = df_nordeste.groupby(['Nome_UF'])['PIB per capita'].describe()
df_nordeste_teste['coef_variacao'] = ((df_nordeste_teste['std']/df_nordeste_teste['mean']) *100).round(2)
df_nordeste_teste = df_nordeste_teste.sort_values(by='coef_variacao',ascending=False)
df_nordeste_teste

**Nota**: o Coeficiente de variação (CV) é a razão entre desvio padrão e média, expressa em %, e serve para comparar a variabilidade relativa, importante para entender desigualdade nesse contexto.

**Análise**: Os estados Coeficiente de variação mais alto têm desigualdade maior entre seus municípios. 
Sendo o top 5: Maranhão, Bahia, Rio Grande do Norte, Pernambuco e Piauí. 

In [ ]:
estados_desiguais = ['Maranhão', 'Bahia', 'Rio Grande do Norte']
df_desiguais = df_nordeste[df_nordeste['Nome_UF'].isin(estados_desiguais)].sort_values(by='PIB per capita', ascending=False)

df_desiguais

In [ ]:
import pandas as pd

# Estados mais desiguais
estados_desiguais = ['Maranhão', 'Bahia', 'Rio Grande do Norte']

# Lista para armazenar os resultados
resultados = []

for estado in estados_desiguais:
    # Filtra só o estado
    df_estado = df_nordeste[df_nordeste['Nome_UF'] == estado]
    
    # Calcula a média do estado
    media_estado = df_estado['PIB per capita'].mean()
    
    # Pega os top 5 municípios por PIB per capita
    top_municipios = df_estado.sort_values(by='PIB per capita', ascending=False).head(5)
    
    # Adiciona ao resultado
    for _, row in top_municipios.iterrows():
        resultados.append({
            'Estado': estado,
            'Município': row['Nome_Município'],
            'PIB per capita': row['PIB per capita'],
            'Média do estado': media_estado,
            'Acima da média (%)': round(((row['PIB per capita'] - media_estado) / media_estado) * 100, 2)
        })

# Converte para DataFrame
df_top = pd.DataFrame(resultados)

# Mostra o resultado ordenado por estado e PIB
df_top = df_top.sort_values(by=['Estado', 'PIB per capita'], ascending=[True, False])


In [ ]:
df_uf_agrupado = df_nordeste.groupby('Nome_UF').sum(numeric_only=True).reset_index()
df_uf_agrupado['PIB per capita (UF)'] = (df_uf_agrupado['PIB (Mil Reais)'] * 1000) /df_uf_agrupado['População (Pessoas)']
df_uf_agrupado = df_uf_agrupado.drop(columns=['PIB per capita'])
df_uf_agrupado

In [ ]:
df_uf_agrupado['% VA da agropecuária'] = ((df_uf_agrupado['VA da agropecuária (Mil Reais)']/df_uf_agrupado['PIB (Mil Reais)'])*100).round(2)
df_uf_agrupado['% VA da indústria'] = ((df_uf_agrupado['VA da indústria (Mil Reais)']/df_uf_agrupado['PIB (Mil Reais)'])*100).round(2)
df_uf_agrupado['% VA dos serviços'] = ((df_uf_agrupado['VA dos serviços (Mil Reais)']/df_uf_agrupado['PIB (Mil Reais)'])*100).round(2)
df_uf_agrupado['% VA da administração pública'] = ((df_uf_agrupado['VA da administração pública (Mil Reais)']/df_uf_agrupado['PIB (Mil Reais)'])*100).round(2)
df_uf_agrupado['% Impostos'] = ((df_uf_agrupado['Impostos (Mil Reais)']/df_uf_agrupado['PIB (Mil Reais)'])*100).round(2)

In [ ]:
df_nordeste_teste = df_nordeste.groupby(['Nome_Município'])['PIB per capita'].describe()
df_nordeste_teste

**Medidas de tendência central e dispersão PIB per capita**

# Gráficos

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df_cv = pd.DataFrame(df_nordeste_teste['coef_variacao'])
df_cv = df_cv.reset_index()

sns.set_theme(style='whitegrid')

plt.figure(figsize=(10,6))
barplot = sns.barplot(x='coef_variacao', y='Nome_UF', data=df_cv, palette='viridis')

plt.title('Coeficiente de Variação do PIB per capita por Estado do Nordeste', fontsize=14)
plt.xlabel('Coeficiente de Variação (%)', fontsize=12)
plt.ylabel('Estado', fontsize=12)

for index, value in enumerate(df_cv['coef_variacao']):
    plt.text(value + 2, index, f'{value:.2f}%', va='center')

# Adiciona legenda explicativa embaixo do gráfico
plt.figtext(0.5, -0.05, 
            "Coeficiente de Variação (CV): mede a variação relativa dos valores em relação à média.\n"
            "CV baixo indica PIB per capita mais homogêneo entre municípios de um estado;\n"
            "CV alto indica maior desigualdade interna no PIB per capita.", 
            wrap=True, horizontalalignment='center', fontsize=10)

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style='whitegrid')
plt.figure(figsize=(12, 6))
barplot = sns.barplot(
    x='PIB per capita', 
    y='Município', 
    hue='Estado', 
    data=df_top, 
    palette='viridis'
)

# Adicionar valores nas barras
for p in barplot.patches:
    valor = p.get_width()
    if valor > 0:
        if valor > df_top['PIB per capita'].max() * 0.85:
            barplot.annotate(
                f'{valor:,.2f}',
                (valor - (valor * 0.05), p.get_y() + p.get_height() / 2),
                ha='right', va='center',
                color='white', fontsize=9, weight='bold'
            )
        else:
            barplot.annotate(
                f'{valor:,.2f}',
                (valor, p.get_y() + p.get_height() / 2),
                ha='left', va='center',
                color='black', fontsize=9
            )

# Linha de média para cada estado + valor na legenda
for estado in estados_desiguais:
    media = df_top[df_top['Estado'] == estado]['Média do estado'].iloc[0]
    plt.axvline(
        media, linestyle='--', 
        label=f'Média {estado} ({media:,.2f})',  # 👈 adiciona o valor no label
        alpha=0.6
    )

# Título e eixos
plt.title('Top 5 Municípios que Mais Elevam o PIB per capita\n(Maranhão, Bahia e Rio Grande do Norte)', fontsize=14)
plt.xlabel('PIB per capita (R$)', fontsize=12)
plt.ylabel('Município', fontsize=12)

# Evita legendas duplicadas
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys(), bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.show()


```
O gráfico acima mostra os munícipios que elevam o PIB per capita dos 3 estados mais desiguais do nordeste. 
Bahia: São Francisco do Conde lidera isoladamente, com quase R$ 300 mil por habitante, puxado pela refinaria de petróleo.

Maranhão: Tasso Fragoso e Balsas se destacam pelo agronegócio de soja.

Rio Grande do Norte: São Bento do Norte tem alta participação energética.
``` 

In [ ]:
#estado
df_ordenado = df_uf_agrupado.sort_values(by='PIB per capita (UF)', ascending=False)
top3 = df_ordenado.head(3)
top3

In [ ]:
bottom3 = df_ordenado.tail(3)
bottom3

In [ ]:
# Medidas de tendência central
media = df_ordenado['PIB per capita (UF)'].mean()
mediana = df_ordenado['PIB per capita (UF)'].median()
moda = df_ordenado['PIB per capita (UF)'].mode()[0]  # Se existir mais de uma, pega a primeira

# Medidas de dispersão
desvio_padrao =df_ordenado['PIB per capita (UF)'].std()
variancia =df_ordenado['PIB per capita (UF)'].var()
amplitude =df_ordenado['PIB per capita (UF)'].max() -df_ordenado['PIB per capita (UF)'].min()
q1 =df_ordenado['PIB per capita (UF)'].quantile(0.25)
q3 =df_ordenado['PIB per capita (UF)'].quantile(0.75)
iqr = q3 - q1

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

fig, axes = plt.subplots(1, 2, figsize=(12,5))

# Top 3
ax1 = sns.barplot(ax=axes[0], x='PIB per capita (UF)', y='Nome_UF', data=top3, palette='Greens_r')
axes[0].set_title('Top 3 PIB per capita agregado por estado')
axes[0].set_xlabel('PIB per capita')

# Adiciona labels com valores nas barras
for container in ax1.containers:
    ax1.bar_label(container, fmt='%.0f', padding=3)  # sem casas decimais, com espaçamento

# Bottom 3
ax2 = sns.barplot(ax=axes[1], x='PIB per capita (UF)', y='Nome_UF', data=bottom3, palette='Reds')
axes[1].set_title('Bottom 3 PIB per capita agregado por estado')
axes[1].set_xlabel('PIB per capita')

for container in ax2.containers:
    ax2.bar_label(container, fmt='%.0f', padding=3)

plt.tight_layout()
plt.show()


In [ ]:
#minicipio
df_ordenado_1 = df_nordeste.sort_values(by='PIB per capita', ascending=False)
top5 = df_ordenado_1.head(5)
top5

In [ ]:
bottom5 = df_ordenado_1.tail(5)
bottom5

In [ ]:
filtro_top3 = df_nordeste[df_nordeste['Nome_UF'].isin(['Bahia','Rio Grande do Norte','Pernambuco'])]
filtro_ba = df_nordeste[df_nordeste['Nome_UF'].isin(['Bahia'])]
filtro_rn = df_nordeste[df_nordeste['Nome_UF'].isin(['Rio Grande do Norte'])]
filtro_pe = df_nordeste[df_nordeste['Nome_UF'].isin(['Pernambuco'])]


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(8,5))
sns.set_style("whitegrid")

ax = sns.barplot(x='PIB per capita', y='Município_x', data=top5, palette='Greens_r')
ax.set_title('Top 5 PIB per capita Municipal')
ax.set_xlabel('PIB per capita')
ax.set_ylabel('Município')

# Adiciona os valores na barra
for container in ax.containers:
    ax.bar_label(container, fmt='%.0f', padding=3)

sns.despine(left=True, bottom=True)
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(8,5))
sns.set_style("whitegrid")

ax = sns.barplot(x='PIB per capita', y='Município_x', data=bottom5, palette='Reds')
ax.set_title('Bottom 5 PIB per capita Municipal')
ax.set_xlabel('PIB per capita')
ax.set_ylabel('Município')

# Adiciona os valores na barra
for container in ax.containers:
    ax.bar_label(container, fmt='%.2f', padding=3)

sns.despine(left=True, bottom=True)
plt.tight_layout()
plt.show()


In [ ]:
df_ordenado_2 = df_nordeste.sort_values(by='PIB (Mil Reais)', ascending=False)
df_ordenado_2

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(8,5))
sns.set_style("whitegrid")  # estilo mais clean, com grid suave

# Gráfico Top 5
ax = sns.barplot(x='PIB (Mil Reais)', y='Município_x', data=df_ordenado_2.head(5), palette='Greens_r')
ax.set_title('Top 5 PIB Municipal')
ax.set_xlabel('PIB')
ax.set_ylabel('Município')

# Remove bordas (spines)
sns.despine(left=True, bottom=True)

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(8,5))
sns.set_style("whitegrid")  # estilo mais clean, com grid suave

# Gráfico Top 5
ax = sns.barplot(x='PIB (Mil Reais)', y='Município_x', data=df_ordenado_2.tail(5), palette='Reds')
ax.set_title('Bottom 5 PIB Municipal')
ax.set_xlabel('PIB')
ax.set_ylabel('Município')

# Remove bordas (spines)
sns.despine(left=True, bottom=True)

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(8,5))
sns.set_style("whitegrid")  # estilo mais clean, com grid suave

# Gráfico Top 5
ax = sns.barplot(x='PIB (Mil Reais)', y='Município_x', data=top5, palette='Greens_r')
ax.set_title('Top 5 PIB per capita Municipal')
ax.set_xlabel('PIB per capita')
ax.set_ylabel('Município')

# Remove bordas (spines)
sns.despine(left=True, bottom=True)

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

estados = df_uf_agrupado['Nome_UF']
pib = df_uf_agrupado['PIB (Mil Reais)']
impostos = df_uf_agrupado['Impostos (Mil Reais)']
agro = df_uf_agrupado['VA da agropecuária (Mil Reais)']
industria = df_uf_agrupado['VA da indústria (Mil Reais)']
servicos = df_uf_agrupado['VA dos serviços (Mil Reais)']
adm_publica = df_uf_agrupado['VA da administração pública (Mil Reais)']

total = pib.values

agro_pct = (agro / total) * 100
industria_pct = (industria / total) * 100
servicos_pct = (servicos / total) * 100
adm_publica_pct = (adm_publica / total) * 100
impostos_pct = (impostos / total) * 100

x = np.arange(len(estados))

fig, ax = plt.subplots(figsize=(14,7))

b1 = ax.bar(x, agro_pct, label='Agropecuária')
b2 = ax.bar(x, industria_pct, bottom=agro_pct, label='Indústria')
b3 = ax.bar(x, servicos_pct, bottom=agro_pct+industria_pct, label='Serviços')
b4 = ax.bar(x, adm_publica_pct, bottom=agro_pct+industria_pct+servicos_pct, label='Administração Pública')
b5 = ax.bar(x, impostos_pct, bottom=agro_pct+industria_pct+servicos_pct+adm_publica_pct, label='Impostos')

ax.set_xticks(x)
ax.set_xticklabels(estados, rotation=45, ha='right')
ax.set_ylabel('Percentual (%)')
ax.set_title('Participação percentual dos setores no PIB por Estado (Nordeste)')

ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

def add_labels(barras):
    for barra in barras:
        altura = barra.get_height()
        if altura > 3:
            ax.text(barra.get_x() + barra.get_width()/2, barra.get_y() + altura/2,
                    f'{altura:.1f}%', ha='center', va='center', color='white', fontsize=9, fontweight='bold')

add_labels(b1)
add_labels(b2)
add_labels(b3)
add_labels(b4)
add_labels(b5)

plt.tight_layout(rect=[0, 0, 0.85, 1])
plt.show()

In [ ]:
df_nordeste["PIB per capita"].skew()

**Assimetria** é uma medida de "inclinação" dos valores em relação a sua média.
**assimetria positiva:** indica que a causa está a direita

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Cria o histograma do PIB per capita (municípios do Nordeste) com log
plt.figure(figsize=(8,5))
plt.hist(np.log10(df_nordeste["PIB per capita"]), bins=60, color="skyblue", edgecolor="black")

# Títulos e rótulos ajustados
plt.title("Distribuição do PIB per capita (log10) - Municípios do Nordeste", fontsize=14, fontweight="bold")
plt.xlabel("Log10 do PIB per capita (R$)", fontsize=12)
plt.ylabel("Número de municípios", fontsize=12)

# Média e mediana no gráfico
media = np.mean(np.log10(df_nordeste["PIB per capita"]))
mediana = np.median(np.log10(df_nordeste["PIB per capita"]))
plt.axvline(media, color='red', linestyle='--', label=f"Média: {media:.2f}")
plt.axvline(mediana, color='green', linestyle=':', label=f"Mediana: {mediana:.2f}")

plt.legend()
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.show()



````
Cada barra representa uma faixa de PIB per capita (em escala logarítmica).
O eixo Y mostra quantos municípios estão naquela faixa.
Ao usar o log10 do PIB per capita, nós transformamos valores que iam de alguns poucos reais por pessoa até centenas de milhares em uma escala mais compacta e interpretável.

Interpretação: A distribuição é fortemente assimétrica (cauda longa à direita). O log(10) suaviza essa assimetria, tornando o padrão mais claro.
Existem alguns poucos municípios com PIB per capita altíssimo (outliers).
A média é puxada para cima por esses outliers, ficando maior que a mediana.
````


In [ ]:
df_nordeste["PIB per capita"].kurtosis()

In [ ]:
df_nordeste["PIB (Mil Reais)"].kurtosis()

**Curtose**: Medida em relação a causa. 
Curtose elevada indica muitos outliers.

**Boxplot**: método gráfico para demonstrar localização, dispersão e asssimetria de dados numéricos.

In [ ]:
import seaborn as sns

sns.set_theme(style="whitegrid")
sns.boxplot(x="PIB per capita", data=df_nordeste, palette="Set2", whis=10)



In [ ]:
q1=df_nordeste["PIB per capita"].quantile(0.25)
q3=df_nordeste["PIB per capita"].quantile(0.75)
m= df_nordeste["PIB per capita"].mean()
iqr = q3 - q1
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr


In [ ]:
import seaborn as sns

sns.set_theme(style="whitegrid")
sns.boxplot(x="PIB (Mil Reais)", data=df_nordeste, palette="Set2", whis=0.5)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

q1 = df_nordeste['PIB (Mil Reais)'].quantile(0.25)
q3 = df_nordeste['PIB (Mil Reais)'].quantile(0.75)
iqr = q3 - q1
limite_inferior = q1 - 1.5 * iqr
limite_superior = q3 + 1.5 * iqr
outliers = (df_nordeste['PIB (Mil Reais)'] < limite_inferior) | (df_nordeste['PIB (Mil Reais)'] > limite_superior)

plt.figure(figsize=(12, 8))
sns.scatterplot(data=df_nordeste, x='População (Pessoas)', y='PIB (Mil Reais)', hue=outliers, palette={True: 'orange', False: 'green'}, alpha=0.7)

plt.title('Relação entre População e PIB total nos Municípios do Nordeste', fontsize=14)
plt.xlabel('População (Pessoas)')
plt.ylabel('PIB (Mil Reais)')
plt.xscale('log')
plt.yscale('log')
plt.legend(title='Outlier', labels=['Sim', 'Não'], bbox_to_anchor=(1.02, 1), loc='upper left')
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

q1 = df_nordeste['PIB per capita'].quantile(0.25)
q3 = df_nordeste['PIB per capita'].quantile(0.75)
iqr = q3 - q1
limite_inferior = q1 - 1.5 * iqr
limite_superior = q3 + 1.5 * iqr
outliers = (df_nordeste['PIB per capita'] < limite_inferior) | (df_nordeste['PIB per capita'] > limite_superior)

plt.figure(figsize=(12, 8))
sns.scatterplot( data=df_nordeste, x='População (Pessoas)', y='PIB per capita', hue=outliers, palette={True: 'orange', False: 'green'}, alpha=0.7)

plt.title('Relação entre População e PIB per capita nos Municípios do Nordeste', fontsize=14)
plt.xlabel('População (Pessoas)')
plt.ylabel('PIB per capita')
plt.xscale('log')
plt.yscale('log')
plt.legend(title='Outlier', labels=['Sim', 'Não'], bbox_to_anchor=(1.02, 1), loc='upper left')
plt.tight_layout()
plt.show()


In [ ]:
df_bahia=df_nordeste[df_nordeste['Nome_UF']=="Bahia"]

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

q1 = df_bahia['PIB per capita'].quantile(0.25)
q3 = df_bahia['PIB per capita'].quantile(0.75)
iqr = q3 - q1
limite_inferior = q1 - 1.5 * iqr
limite_superior = q3 + 1.5 * iqr
outliers = (df_bahia['PIB per capita'] < limite_inferior) | (df_bahia['PIB per capita'] > limite_superior)

plt.figure(figsize=(12, 8))
sns.scatterplot( data=df_bahia, x='População (Pessoas)', y='PIB per capita', hue=outliers, palette={True: 'orange', False: 'green'}, alpha=0.7)

plt.title('Relação entre População e PIB per capita nos Municípios do Nordeste', fontsize=14)
plt.xlabel('População (Pessoas)')
plt.ylabel('PIB per capita')
plt.xscale('log')
plt.yscale('log')
plt.legend(title='Outlier', labels=['Sim', 'Não'], bbox_to_anchor=(1.02, 1), loc='upper left')
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np

df_nordeste["LOG10 Pib Per Capta"] = (np.log10(df_nordeste["PIB per capita"]))

In [ ]:
import seaborn as sns

sns.set_theme(style="whitegrid")
sns.boxplot(x="LOG10 Pib Per Capta", data=df_nordeste, palette="Set2", whis=0.5)



In [ ]:
data = df_nordeste[df_nordeste["Nome_UF"] == "Maranhão"]

data["LOG10 Pib Per Capita"] = (np.log10(data["PIB per capita"]))

data

In [ ]:

import seaborn as sns

sns.set_theme(style="darkgrid")
sns.boxplot(x="LOG10 Pib Per Capita", data=data, palette="Set2", whis=0.5)




In [ ]:
data = df_nordeste[df_nordeste["Nome_UF"] == "Ceará"]

data["LOG10 Pib Per Capita"] = (np.log10(data["PIB per capita"]))

data

In [ ]:

import seaborn as sns

sns.set_theme(style="ticks")
sns.boxplot(x="LOG10 Pib Per Capita", data=data, palette="Set2", whis=0.5)

